In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import datetime


In [2]:
dataset=pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]

In [8]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [9]:
dependent=dataset[['charges']]

In [10]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)  


In [12]:
X_train,X_test,Y_train,Y_test

(      age     bmi  children  sex_male  smoker_yes
 1163   18  28.215         0         0           0
 196    39  32.800         0         0           0
 438    52  46.750         5         0           0
 183    44  26.410         0         0           0
 1298   33  27.455         2         1           0
 ...   ...     ...       ...       ...         ...
 763    27  26.030         0         1           0
 835    42  35.970         2         1           0
 1216   40  25.080         0         1           0
 559    19  35.530         0         1           0
 684    33  18.500         1         0           0
 
 [936 rows x 5 columns],       age     bmi  children  sex_male  smoker_yes
 578    52  30.200         1         1           0
 610    47  29.370         1         0           0
 569    48  40.565         2         1           1
 1034   61  38.380         0         1           0
 198    51  18.050         0         0           0
 ...   ...     ...       ...       ...         ...
 1261

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_boston
param_grid = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

grid = GridSearchCV(XGBRegressor(), param_grid, refit = True, verbose=3,n_jobs=-1)

# fitting the model for grid search
grid.fit(X_train, Y_train)


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   21.6s finished


[16:45:51] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[16:45:51] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    le...
                                    predictor=None, random_state=None,
                                    reg_alpha=None, ...),
             iid='warn', n_jobs=-1,
             param_grid={'colsample_bytree': [0.7],
                         'learning_rate': [0.03, 0.05, 0.07],
              

In [15]:
re=grid.cv_results_
print("The R_score value for best parameter{}:".format(grid.best_params_))

The R_score value for best parameter{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}:


In [16]:
table=pd.DataFrame.from_dict(re)

In [17]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_min_child_weight,param_n_estimators,param_nthread,param_objective,param_silent,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,2.262816,0.062018,0.032981,2.942492e-03,0.7,0.03,5,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.796086,0.814298,0.803389,0.804591,0.007483,1
1,2.667871,0.008140,0.025651,2.355559e-03,0.7,0.03,6,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.786390,0.802106,0.798547,0.795681,0.006729,2
2,3.096095,0.113424,0.022337,1.241417e-03,0.7,0.03,7,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.772273,0.793159,0.791377,0.785603,0.009454,4
3,2.284594,0.029724,0.021653,4.716514e-04,0.7,0.05,5,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.776127,0.800588,0.797901,0.791539,0.010953,3
4,2.604062,0.026546,0.021987,8.156588e-04,0.7,0.05,6,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.759903,0.791213,0.786815,0.779310,0.013840,6
5,2.451156,0.184792,0.022320,4.712580e-04,0.7,0.05,7,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.747501,0.779796,0.782728,0.770008,0.015960,8
6,1.914487,0.016347,0.020987,2.973602e-07,0.7,0.07,5,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.765793,0.788590,0.786472,0.780285,0.010283,5
7,2.028750,0.006373,0.022320,4.710333e-04,0.7,0.07,6,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.747041,0.782034,0.785246,0.771440,0.017303,7
8,1.973451,0.048552,0.023652,4.710342e-04,0.7,0.07,7,4,500,4,reg:linear,1,0.7,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",0.735146,0.770050,0.772062,0.759086,0.016948,9


In [18]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age:32
BMI:43
Children:2
Sex Male 0 or 1:0
Smoker Yes 0 or 1:1


In [19]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[51619.25]
